<a href="https://colab.research.google.com/github/abhranil-datascience/CompressPDF/blob/master/PDFCompressionNoteBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
######################### Suppress Warnings #######################################
import warnings
warnings.filterwarnings('ignore')
############################## Mount Drive ########################################
from google.colab import drive
drive.mount('/content/gdrive')
############################## Change Directory ###################################
import os
os.chdir('/content/gdrive/My Drive/PdfImagePath')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
############################################## Install Required Library #################################################
!pip install --upgrade pymupdf
!pip install wand
!apt-get install libmagickwand-dev

     |████████████████████████████████| 5.3MB 4.4MB/s 
     |████████████████████████████████| 133kB 4.8MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gir1.2-freedesktop
  gir1.2-gdkpixbuf-2.0 gir1.2-rsvg-2.0 gsfonts imagemagick-6-common
  libcairo-script-interpreter2 libcairo2-dev libcupsfilters1 libcupsimage2
  libdjvulibre-dev libdjvulibre-text libdjvulibre21 libgdk-pixbuf2.0-dev
  libgs9 libgs9-common libijs-0.35 libjbig2dec0 liblcms2-dev liblqr-1-0
  liblqr-1-0-dev libmagickcore-6-arch-config libmagickcore-6-headers
  libmagickcore-6.q16-3 libmagickcore-6.q16-3-extra libmagickcore-6.q16-dev
  libmagickwand-6-headers libmagickwand-6.q16-3 libmagickwand-6.q16-dev
  libpixman-1-dev librsvg2-dev libwmf-dev libwmf0.2-7 libxcb-shm0-dev
  poppler-data
Suggested packages:
  fonts-noto ghostscript-x libcairo2-doc inkscape libjxr-tool

In [0]:
############################################# Change Cache And Permission Policies For ImageMagick #######################################
!sed -i 's/none/read|write/g' /etc/ImageMagick-6/policy.xml
!sed -i 's/256MiB/2GiB/g' /etc/ImageMagick-6/policy.xml

In [0]:
####################################### Reboot Runtime To Reflect The Above Policy File Changes ############################################
import os
os.kill(os.getpid(), 9)
######################################### Runtime Rebooted Now Run All Cells After this cell ################################################

In [0]:
################################################# Define Constants #####################################################
InputFileName='samplefile.pdf'
InputFilePath='/content/gdrive/My Drive/PdfImagePath/OriginalPDF/'
InputFileFullPath=InputFilePath+InputFileName

ExtractedImagesDirectory='/content/gdrive/My Drive/PdfImagePath/Shrinked/'

FinalPDFDirectory='/content/gdrive/My Drive/PdfImagePath/FinalPDF/'
CompressedPDFName='CompressedPDF.pdf'
FinalPDFFullPath=FinalPDFDirectory+CompressedPDFName


In [0]:
########################################## Count Number Of Pages In Original PDF #########################################
import fitz
doc = fitz.open(InputFileFullPath) 
Num_Of_Pages_In_Original_Document=doc.pageCount
print("Number Of Pages In Original Document: "+str(Num_Of_Pages_In_Original_Document))

Number Of Pages In Original Document: 32


In [0]:
#################################### Size Of Original Document ################################################
import os
statinfo = os.stat(InputFileFullPath)
SizeInMBOriginalDocument=statinfo.st_size/1e+6
print("Size Of Original PDF: "+str(SizeInMBOriginalDocument)+" MB ")

Size Of Original PDF: 3.435045 MB 


In [0]:
############################### Convert PDF Pages To PNG ####################################################
from wand.image import Image
from wand.color import Color
ImagePaths=[]
all_pages = Image(filename=InputFileFullPath)
for i, page in enumerate(all_pages.sequence):
  with Image(page) as img:
    img.format = 'png'
    img.background_color = Color('white')
    img.alpha_channel = 'remove'
    img.compression_quality = 90
    shrinked_image_name='Image-'+str(i+1)+".png"
    shrinked_image_full_path=ExtractedImagesDirectory+shrinked_image_name
    ImagePaths.append(shrinked_image_full_path)
    img.save(filename=shrinked_image_full_path)

In [0]:
####################################################### Create New PDF Document Containing New Compressed Images #######################################################
CompressedDocument=fitz.open()
for image in ImagePaths:
  imgdoc=fitz.open(image)
  pdfbytes=imgdoc.convertToPDF()
  imgpdf=fitz.open("pdf", pdfbytes)
  CompressedDocument.insertPDF(imgpdf)
CompressedDocument.save(FinalPDFDirectory+CompressedPDFName)
os.chdir(ExtractedImagesDirectory)
! rm *

In [0]:
#################################### Size Of Final Document ################################################
statinfo = os.stat(FinalPDFFullPath)
SizeInMBCompressedDocument=statinfo.st_size/1e+6
print("Size Of Compressed PDF: "+str(SizeInMBCompressedDocument)+" MB ")

Size Of Compressed PDF: 1.957331 MB 


In [0]:
################################## Compression Ratio #####################################################
Ratio=((SizeInMBOriginalDocument-SizeInMBCompressedDocument)/SizeInMBOriginalDocument)*100
print("Document Compressed by "+str(int(Ratio))+"% !!")

Document Compressed by 43% !!
